In [ ]:
import mindpype as mp
import pkgutil, numpy as np

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline.

In [ ]:
# Create the pipeline using the factory method
session = mp.Session.create()

Following the creation of the pipeline, we need to create the graphs, which are the representations of the pipeline in MindPype.

Each distinct pipeline would require a different graph object. For this tutorial, we will only require a single graph.

In [ ]:
# Create the graph using the factory method
graph = mp.Graph.create(session)

With a graph created, we can now attach our input and output data containers. Since the input data is known, we can create a tensor (the data container in MindPype), using the ```create_from_data()``` factory method. For the output tensor, the library will automatically determine the correct size for the tensor, so we can create a "virtual" tensor, meaning that it can be modified by the library and does not contain any user provided data.       

In [ ]:
# Create a random input tensor using the factory method
input_tensor = mp.Tensor.create_from_data(session, shape=(10, 10), data=np.random.rand(10, 10))

# Create an output tensor using the factory method
output_tensor = mp.Tensor.create(session, shape=(10, 10))


With the input and output tensors, we can know add the processing nodes to the pipeline. For this example, we are using a single processing step, or kernel; the baseline correction kernel. We can use the following factory method to add  the kernel to a node, and add the node to the graph we created previously.

Futhermore, we must pass the input and output tensors, as well as the required parameters for the kernel (in this case, the baseline period), to the factory method shown below.

In [ ]:
# Create the baseline node, passing the graph, input tensor, output tensor, and baseline period
baseline_node = mp.kernels.BaselineCorrectionKernel.add_baseline_node(graph, input_tensor, output_tensor, baseline_period = [0, 10])

At this point we have created the processing pipeline, and added all of the required components. We can now use a graph method to verify that the structure of the pipeline is valid.

In [ ]:
# Verify the structure and inputs/outputs of the graph
graph.verify()


Following the verification of the pipeline, we should now initialize the graph. This step is required for pipelines that have methods that need to be trained or fit (ie. classifiers), but optional for other pipelines.

In [ ]:
# Initialize the graph - optional for this pipeline
graph.initialize()

The graph is now ready to execute the provided input data. For other paradigms, we may opt to execute the graph in a loop. If we were using class-separated data, we would need to pass a task label to poll the correct task.

In [ ]:
# Execute the graph
graph.execute()



In [ ]:
# Visualize the input and output data
print(input_tensor.data)
print(output_tensor.data)

In [ ]:
# Confirm proper functionality
corrected = input_tensor.data - np.mean(input_tensor.data, axis = -1, keepdims = True)

assert np.allclose(output_tensor.data, corrected)